# <center>How it works</center>

Certain web pages try to restrict the possibility of being visited by robots creating content that shows on clients by a dinamic way (trough Javascript code), like [milanuncios.com](https://www.milanuncios.com/).

### Static scraping in Python


In [1]:
import requests
from bs4 import BeautifulSoup

url = "https://www.milanuncios.com/anuncios/masajes-relajantes-en-jerez.htm"

soup = BeautifulSoup(requests.get(url).content, "html.parser")
print(soup.find(class_="aditem-detail-title"))

None


Although, is very easy avoid these limitations...

### Dinamic scraping in Python

In [2]:
from pyvirtualdisplay import Display  # pip3 install pyvirtualdisplay
from selenium import webdriver        # pip3 install selenium
from bs4 import BeautifulSoup         # pip3 install bs4
import time

url = "https://www.milanuncios.com/anuncios/masajes-relajantes-en-jerez.htm"

display = Display(visible=0, size=(800, 600))
display.start()
browser = webdriver.Firefox()  # You need geckodriver and Mozilla Firefox
browser.get(url)
time.sleep(.8)
soup = BeautifulSoup(browser.page_source, "html.parser")
print(soup.find(class_="aditem-detail-title"))
browser.quit()

<a class="aditem-detail-title" href="/masajistas/masajes-y-tratamientos-fisio-domicilio-246582919.htm" target="_blank"><b class="sub2">MASAJES</b> Y TRATAMIENTOS FISIO DOMICILIO</a>


________________________________________________

# <center>Documentation</center>

## Basic usage
The 4 main methods to realize most of queries in are the next:

In [ ]:
from milanuncios import MilAnuncios

# We must enter to milanuncios like context (with statement)
with MilAnuncios() as ma:

    # Obtain main categories of home page:
    print(ma.categories) 

    # Obtain subcategories (and subsubcategories recursively) from a main category:
    print(ma.subcategories("servicios"))

    # Realize a query (we indicates number of pages, 1 page as default):
    print(ma.search("gatos", pages=3)) # Returns a pandas' DataFrame

    # Realize a search by category/sucategory:
    print(ma.search_category("juegos", subcategory="videoconsolas").tail())

### Ads renewal

In [ ]:
from milanuncios import MilAnunciosLoginError

# delay parameter indicates how many seconds to wait loading pages and before perform actions
# Is 1.5 as default, if you have runtime troubles try to increase it
with MilAnuncios(delay=3) as ma:
    # Login in milanuncios
    ma.login("tu_email@proveedor.com", "tu_contraseña")  # If login fails MilAnunciosLoginError will be raised
    assert ma.logged == True
    
    # Obtain your ads
    ma.my_ads(dataframe=False) # As default returns a pandas' DataFrame, but you can retrieve a list also
    
    # Renew your ads
        # You can renew by title of by number of adverts
        # The program will ignore that adverts wich can't be renewed yet
    ma.renew_ads(title=["Título de mi anuncio", "Otro, da igual si es en minúscula o mayúscula"])  # Por nombre
    
    ma.renew_ads(number=3)  # First 3 that can be renewed in your adverts list
    
    # This method returns the number of adverts renewed

You also can obtain or renew your ads without execute `login()` method, only pass it your email and password to ` my_ads()` o `renew_ads()` as two first parameters:

In [ ]:
options = {}
#options = dict(debug=True)  # If you want to display on screen renovation process activate debug

with MilAnuncios(**options) as ma:
    ma.my_ads("tu_email@proveedor.com", "tu_contraseña")
    
    # If you don't pass number or titles list parameters will be renewed all adverts
    ma.renew_ads("tu_email@proveedor.com", "tu_contraseña")       


_____________________________
## Advanced usage
### Filters

In [ ]:
# We can filter by regions in both methods:
print(ma.search("guitarra flamenca", region="cadiz").head())
# To consult all possible regions (provinces and autonomous communities):
print(ma.regions)

# If you want to search only by regions (https://www.milanuncios.com/anuncios-en-sevilla/)
print(ma.search("", region="sevilla").head())  # Void string in query parameter

# We can filter by offer and demand. As default, offer and demand paramters are True:
print(ma.search_category("inmobiliaria", subcategory="alquiler de casas", offer=False))

### Caching
If you are going to make a lot of queries in only one session, it is convenient to caching all subcategories when instantiating the scraper class. It will take some time storing in memory all subcategories, but then the performance increases:

In [ ]:
from pprint import pprint

ma = MilAnuncios(init_cache=True)
pprint(ma.cache["categories"])
pprint(ma.cache["subcategories"])